In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Spark book")\
.getOrCreate()

Error: Missing application resource.

Usage: spark-submit [options] <app jar | python file | R file> [app arguments]
Usage: spark-submit --kill [submission ID] --master [spark://...]
Usage: spark-submit --status [submission ID] --master [spark://...]
Usage: spark-submit run-example [options] example-class [example args]

Options:
  --master MASTER_URL         spark://host:port, mesos://host:port, yarn,
                              k8s://https://host:port, or local (Default: local[*]).
  --deploy-mode DEPLOY_MODE   Whether to launch the driver program locally ("client") or
                              on one of the worker machines inside the cluster ("cluster")
                              (Default: client).
  --class CLASS_NAME          Your application's main class (for Java / Scala apps).
  --name NAME                 A name of your application.
  --jars JARS                 Comma-separated list of jars to include on the driver
                              and executor classpaths

RuntimeError: Java gateway process exited before sending its port number

In [4]:
spark.conf.set("spark.sql.shuffle.partitions", 5)
static = spark.read.json("../data/activity-data")
streaming = spark\
.readStream\
.schema(static.schema)\
.option("maxFilesPerTrigger", 10)\
.json("../data/activity-data")
streaming.printSchema()

NameError: name 'spark' is not defined

In [3]:
withEventTime = streaming\
.selectExpr("*","cast(cast(Creation_Time as double)/1000000000 as timestamp) as event_time")

NameError: name 'streaming' is not defined

In [42]:
from pyspark.sql.functions import window, col
withEventTime.groupBy(window(col("event_time"), "10 minutes")).count()\
.writeStream\
.queryName("pyevents_per_window")\
.format("memory")\
.outputMode("complete")\
.start()

22/12/17 18:21:07 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4c667052-c20d-4cff-bc39-eeb8b6c08ef4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/12/17 18:21:07 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [46]:
spark.sql("SELECT * FROM pyevents_per_window").show()


+--------------------+------+
|              window| count|
+--------------------+------+
|{2015-02-24 18:50...|150773|
|{2015-02-24 20:00...|133323|
|{2015-02-23 19:30...|100853|
|{2015-02-23 17:20...| 99178|
|{2015-02-24 19:30...|125679|
|{2015-02-24 20:10...|105494|
|{2015-02-23 17:30...|100443|
|{2015-02-23 17:40...| 88681|
|{2015-02-23 20:20...|106075|
|{2015-02-22 07:40...|    35|
|{2015-02-24 18:20...|113768|
|{2015-02-24 19:20...|133623|
|{2015-02-24 21:00...|150225|
|{2015-02-24 21:10...|169064|
|{2015-02-24 20:40...|132243|
|{2015-02-24 20:50...| 96023|
|{2015-02-23 21:30...| 94669|
|{2015-02-23 20:40...|167565|
|{2015-02-24 19:00...|200133|
|{2015-02-23 19:20...|106291|
+--------------------+------+
only showing top 20 rows



In [48]:
from pyspark.sql.functions import window, col
withEventTime.groupBy(window(col("event_time"), "10 minutes", "5 minutes"))\
.count()\
.writeStream\
.queryName("pyevents_per_window_1")\
.format("memory")\
.outputMode("complete")\
.start()

22/12/17 18:26:26 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-39544f9c-7b30-48a4-9f85-c891a8370c47. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/12/17 18:26:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [49]:
spark.sql("SELECT * FROM pyevents_per_window_1").show()


+--------------------+------+
|              window| count|
+--------------------+------+
|{2015-02-23 21:15...|107668|
|{2015-02-24 18:50...|150773|
|{2015-02-24 20:00...|133323|
|{2015-02-22 07:35...|    35|
|{2015-02-23 19:30...|100853|
|{2015-02-23 17:20...| 99178|
|{2015-02-23 20:25...| 91684|
|{2015-02-24 21:25...|203945|
|{2015-02-23 19:55...|113953|
|{2015-02-22 07:40...|    35|
|{2015-02-23 19:35...| 91221|
|{2015-02-23 20:05...|205912|
|{2015-02-24 18:20...|113768|
|{2015-02-24 20:35...|174863|
|{2015-02-24 21:00...|150225|
|{2015-02-24 19:30...|125679|
|{2015-02-24 20:10...|105494|
|{2015-02-23 17:30...|100443|
|{2015-02-24 18:45...|138413|
|{2015-02-23 17:40...| 88681|
+--------------------+------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import window, col
withEventTime\
.withWatermark("event_time", "30 minutes")\
.groupBy(window(col("event_time"), "10 minutes", "5 minutes"))\
.count()\
.writeStream\
.queryName("pyevents_per_window")\
.format("memory")\
.outputMode("complete")\
.start()